In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import gc
import warnings

warnings.filterwarnings('ignore')

In [2]:
def get_psi(train,test,f_cols):
    psi_res = pd.DataFrame()
    psi_dict={}
    for c in tqdm(f_cols):
        try:
            t_train = train[c].fillna(-998)
            t_test = test[c].fillna(-998)
            #获取切分点
            bins=[]
            for i in np.arange(0,1.1,0.2):
                bins.append(t_train.quantile(i))
            bins=sorted(set(bins))
            bins[0]=-np.inf
            bins[-1]=np.inf
            #计算psi
            t_psi = pd.DataFrame()
            t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
            t_psi['test'] = pd.cut(t_test,bins).value_counts()
            if c == 'outdoorTemp':
                print(t_psi['train'])
                print(t_psi['test'])
            t_psi.index=[str(x) for x in t_psi.index]
            t_psi.loc['总计',:] = t_psi.sum()
            t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
            t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
            t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
            t_psi.loc['总计','psi'] = t_psi['psi'].sum()
            t_psi.index.name=c
            #汇总
            t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                                 columns=['变量名','PSI'])
            psi_res = pd.concat([psi_res,t_res])
            psi_dict[c]=t_psi
            print(c,'done')
        except:
            print(c,'error')
    return psi_res,psi_dict

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.DataFrame(test['time'])

In [4]:
train = train[train['temperature'].notnull()]
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')

In [5]:
train.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                 'indoorHum', 'indoorAtmo', 'temperature']

test.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec',
                'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                'indoorHum', 'indoorAtmo']

In [6]:
data = pd.concat([train, test])

In [7]:
data['indoorAtmo-outdoorAtmo'] = data['indoorAtmo'] - data['outdoorAtmo']
data['indoorHum-outdoorHum'] = data['indoorHum'] - data['outdoorHum']

In [8]:
numerical_features = ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum']

In [9]:
for i in numerical_features:
    data['{}_diff'.format(i)] = data[i].diff()

In [10]:
group_feats = []
for i in tqdm(numerical_features):
    data['MDH_{}_mean'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('mean')
    data['MDH_{}_median'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('median')
    data['MDH_{}_max'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('max')
    data['MDH_{}_min'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('min')
    data['MDH_{}_std'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('std')
    data['MDH_{}_skew'.format(i)] = data.groupby(['month', 'day', 'hour'], as_index=False)[i].transform('skew')
    
    group_feats.append('MDH_{}_mean'.format(i))
    group_feats.append('MDH_{}_median'.format(i))

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.37s/it]


In [11]:
# 基本交叉特征
for f1 in tqdm(numerical_features + group_feats):
    for f2 in tqdm(numerical_features + group_feats):
        if f1 != f2:
            colname = '{}_{}_ratio'.format(f1, f2)
            data[colname] = data[f1].values / data[f2].values

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1907.62it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1619.27it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1749.01it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1907.70it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1913.79it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1914.33it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1914.17it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1749.61it/s]

100%|███████████████████████████

In [12]:
data.fillna(method='bfill', inplace=True)

In [13]:
# 历史信息提取
# dt 14-44
data['dt'] = data['day'].values + (data['month'].values - 3) * 31

for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorAtmo-outdoorAtmo', 'indoorHum-outdoorHum', 'temperature']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data[data['dt'] < t].groupby(['hour'])[f].agg({'mean'}).reset_index()
        tmp.columns = ['hour', 'hit_{}_mean'.format(f)]
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)

    data = data.merge(tmp_df, on=['dt', 'hour'], how='left')
    del tmp, tmp_df
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:01<00:00, 21.18it/s]


In [14]:
data.fillna(method='bfill', inplace=True)

In [15]:
# 离散化
for f in numerical_features:
    data[f + '_20_bin'] = pd.cut(data[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_50_bin'] = pd.cut(data[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_100_bin'] = pd.cut(data[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data[f + '_200_bin'] = pd.cut(data[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [16]:
for f1 in tqdm(['{}_20_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')
        data['{}_{}_std'.format(f1, f2)] = data.groupby([f1])[f2].transform('std')
        data['{}_{}_sum'.format(f1, f2)] = data.groupby([f1])[f2].transform('sum')
        data['{}_{}_skew'.format(f1, f2)] = data.groupby([f1])[f2].transform('skew')

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:47<00:00,  6.74s/it]


In [17]:
for f1 in tqdm(['{}_50_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')
        data['{}_{}_std'.format(f1, f2)] = data.groupby([f1])[f2].transform('std')
        data['{}_{}_sum'.format(f1, f2)] = data.groupby([f1])[f2].transform('sum')
        data['{}_{}_skew'.format(f1, f2)] = data.groupby([f1])[f2].transform('skew')

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:12<00:00, 10.35s/it]


In [18]:
for f1 in tqdm(['{}_100_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')
        data['{}_{}_std'.format(f1, f2)] = data.groupby([f1])[f2].transform('std')
        data['{}_{}_sum'.format(f1, f2)] = data.groupby([f1])[f2].transform('sum')
        data['{}_{}_skew'.format(f1, f2)] = data.groupby([f1])[f2].transform('skew')

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:37<00:00, 13.88s/it]


In [19]:
for f1 in tqdm(['{}_200_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        data['{}_{}_median'.format(f1, f2)] = data.groupby([f1])[f2].transform('median')
        data['{}_{}_mean'.format(f1, f2)] = data.groupby([f1])[f2].transform('mean')
        data['{}_{}_max'.format(f1, f2)] = data.groupby([f1])[f2].transform('max')
        data['{}_{}_min'.format(f1, f2)] = data.groupby([f1])[f2].transform('min')
        data['{}_{}_std'.format(f1, f2)] = data.groupby([f1])[f2].transform('std')
        data['{}_{}_sum'.format(f1, f2)] = data.groupby([f1])[f2].transform('sum')
        data['{}_{}_skew'.format(f1, f2)] = data.groupby([f1])[f2].transform('skew')

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:02<00:00, 17.53s/it]


In [20]:
drop_columns = ['time', 'year', 'sec', 'temperature']

train_count = train.shape[0]
train = data[:train_count].copy().reset_index(drop=True)
test = data[train_count:].copy().reset_index(drop=True)
del data
gc.collect()

49

In [21]:
features = train[:1].drop(drop_columns, axis=1).columns
x_train = train[features]
x_test = test[features]

y_train = train['temperature'].values - train['outdoorTemp'].values

In [22]:
psi_res, psi_dict = get_psi(x_train,x_test,features)
features = list(psi_res[psi_res['PSI'] <= 0.2]['变量名'].values) + ['outdoorTemp']

  0%|▎                                                                                | 8/1889 [00:00<00:49, 37.86it/s]

month done
day done
hour done
min done
(-inf, 12.72]    4962
(12.72, 15.0]    5155
(15.0, 16.9]     4914
(16.9, 19.7]     4922
(19.7, inf]      4854
Name: train, dtype: int64
(-inf, 12.72]      0
(12.72, 15.0]     53
(15.0, 16.9]      64
(16.9, 19.7]      64
(19.7, inf]      225
Name: test, dtype: int64
outdoorTemp done
outdoorHum done
outdoorAtmo done
indoorHum done


  1%|▋                                                                               | 17/1889 [00:00<00:47, 39.17it/s]

indoorAtmo done
indoorAtmo-outdoorAtmo done
indoorHum-outdoorHum done
outdoorTemp_diff done
outdoorHum_diff done
outdoorAtmo_diff done
indoorHum_diff done
indoorAtmo_diff done
indoorAtmo-outdoorAtmo_diff done


  1%|▉                                                                               | 22/1889 [00:00<00:46, 39.82it/s]

...

indoorHum-outdoorHum_200_bin_indoorAtmo-outdoorAtmo_max done
indoorHum-outdoorHum_200_bin_indoorAtmo-outdoorAtmo_min done
indoorHum-outdoorHum_200_bin_indoorAtmo-outdoorAtmo_std done
indoorHum-outdoorHum_200_bin_indoorAtmo-outdoorAtmo_sum done
indoorHum-outdoorHum_200_bin_indoorAtmo-outdoorAtmo_skew done
indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_median done


100%|█████████████████████████████████████████████████████████████████████████████▉| 1886/1889 [00:46<00:00, 23.53it/s]

indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_mean done
indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_max done
indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_min done
indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_std done
indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_sum done


100%|██████████████████████████████████████████████████████████████████████████████| 1889/1889 [00:46<00:00, 40.26it/s]

indoorHum-outdoorHum_200_bin_indoorHum-outdoorHum_skew done


In [23]:
x_train = train[features]
x_test = test[features]

In [24]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)
params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.1,
    'seed': 2020,
    'nthread': 36,
    'silent': True,
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model = xgb.train(params,
                  train_matrix,
                  num_boost_round=50000,
                  evals=watchlist,
                  verbose_eval=500,
                  early_stopping_rounds=1000)
val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)
test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

[0]	train-mae:0.57938	eval-mae:0.31365
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 1000 rounds.
[500]	train-mae:0.01870	eval-mae:0.17339
[1000]	train-mae:0.01001	eval-mae:0.17389
Stopping. Best iteration:
[83]	train-mae:0.05154	eval-mae:0.17178



In [25]:
sub["temperature"] = test_pred[:, 0] + test['outdoorTemp'].values
sub.to_csv('../sub/sub_baseline_psi.csv', index=False)